In [10]:
import tensorflow as tf
import numpy as np
import os

# ========== Step 1: Load Dataset ==========
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)

text = open(path_to_file, 'r', encoding='utf-8').read()
print(f"Length of text: {len(text)} characters")

# ========== Step 2: Preprocess Data ==========
vocab = sorted(set(text))
vocab_size = len(vocab)

char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

# ========== Step 3: Create Sequences ==========
seq_length = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)

# ========== Step 4: Batch and Shuffle ==========
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# ========== Step 5: Build the Model ==========
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(None,), batch_size=batch_size))
    model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
    model.add(tf.keras.layers.GRU(
        rnn_units,
        return_sequences=True,
        stateful=True,
        recurrent_initializer='glorot_uniform'
    ))
    model.add(tf.keras.layers.Dense(vocab_size))
    return model

model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)
model.summary()

# ========== Step 6: Compile and Train ==========
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

EPOCHS = 1
history = model.fit(dataset, epochs=EPOCHS)

# ========== Step 7: Save Weights ==========
model.save_weights('textgen.weights.h5')

# ========== Step 8: Rebuild for Generation ==========
gen_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
gen_model.load_weights('textgen.weights.h5')
gen_model.build(tf.TensorShape([1, None]))

# ========== Step 9: Generate Text ==========
def generate_text(model, start_string, num_generate=500, temperature=1.0):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    # ✅ Reset GRU layer state
    model.layers[1].reset_states()

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature

        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# ========== Step 10: Try It Out ==========
output = generate_text(gen_model, start_string="Once upon a time ")
print("\nGenerated Text:\n")
print(output)


Length of text: 1115394 characters


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (64, None, 256)        │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ (64, None, 1024)       │     3,938,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (64, None, 65)         │        66,625 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,021,569 (15.34 MB)

 Trainable params: 4,021,569 (15.34 MB)

 Non-trainable params: 0 (0.00 B)

172/172 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - loss: 3.1139

Generated Text:

Once upon a time woo, I godd geaiser, ne glom
And, theg kile of take m grock fur E
Yours will will youll dourd fort exome he a knaint.
Heeply velies fros padle my gein:
Now frowe your of hivhere?
St pre coodth! Ifran, and batn sead
me hare ondent hat shall ve blece.

QUEZHAUMEO:
I to lead vears liquk, of stly burgus ip not.

SORNLARUMHARD:
I peacht my it so lows,
Het the iceod ar pror that my quhe, aycormar:
An sray makn to the grord;
Hawenm matons chore, bealian a cood ole
and? Buettht Lomanich, suthel, ignand 
